In [1]:
import numpy as np
import os
import cv2
from keras.optimizers import SGD
import keras.backend as K
import scipy.io as sio
from keras.models import Model
# We only test DenseNet-121 in this script for demo purpose
from densenet121 import DenseNet

In [2]:
weights_path = 'densenet121_weights_tf.h5'
model = DenseNet(reduction=0.5, classes=1000, weights_path=weights_path)
model.summary()

In [3]:
#uncomment the line to extract partciluar features

# part_model = Model(inputs=model.input, outputs=model.get_layer('relu5_blk').output)
part_model = Model(inputs=model.input, outputs=model.get_layer('pool5').output)
sgd = SGD(lr=1e-2, decay=1e-6, momentum=0.9, nesterov=True)
part_model.compile(loss='categorical_crossentropy', optimizer=sgd, metrics=['accuracy'])

In [3]:
#text file containing names of images
lines = [line.rstrip('\n') for line in open('I:/split_images_filenames.txt')]

for i in range(0,len(lines)):
    print(i)
    line = lines[i].split(' ')
    image_path = os.path.join('I:/Split_Images/', line[0] + '.tif') 

    img = cv2.imread(image_path)
    test_image = np.zeros((1,img.shape[0],img.shape[1],3))
    test_image[0,:,:,:] = img[:,:,:]
    
    #saving the feature map
    fmap_path = os.path.join('I:/Split_Images_feats/pool5',line[0] +'.mat')
    feature_map = part_model.predict(test_image)
    sio.savemat(fmap_path, {'fmap' : feature_map})